# 📧 Mistral Email Asistanı - Colab Başlatıcı

Bu notebook, projeyi Google Colab'ın T4 GPU'su üzerinde çalıştırmak için hazırlanmıştır.

### Adımlar:
1.  Yukarıdaki menüden **Runtime > Change runtime type** seçin ve **T4 GPU** olduğundan emin olun.
2.  Aşağıdaki **Ayarlar** hücresine Ngrok Token'ınızı ve GitHub Repo adresinizi yapıştırın.
3.  (Opsiyonel) Verilerinizi kaybetmemek için Google Drive ayarını aktif edin.
4.  **Tümünü Çalıştır** (Runtime > Run all) diyerek arkanıza yaslanın.

In [ ]:
# @title ⚙️ Ayarlar
NGROK_AUTHTOKEN = "BURAYA_NGROK_TOKEN_YAPISTIRIN" # @param {type:"string"}
REPO_URL = "https://github.com/KULLANICI_ADI/REPO_ADI.git" # @param {type:"string"}

# Verileri Google Drive'da saklamak istiyorsaniz isaretleyin
USE_GOOGLE_DRIVE = True # @param {type:"boolean"}

if NGROK_AUTHTOKEN == "BURAYA_NGROK_TOKEN_YAPISTIRIN":
    print("❌ Lütfen geçerli bir Ngrok Token girin!")
else:
    print("✅ Ayarlar alındı.")

In [ ]:
# @title 1. Kurulum ve Hazırlık (2-3 dk)
import os
from google.colab import drive

# ---------------------------------------------------------
# 1. Projeyi Çek veya Güncelle
# ---------------------------------------------------------
repo_name = "mistral_app" # Default fallback

if REPO_URL != "https://github.com/KULLANICI_ADI/REPO_ADI.git":
    repo_name = REPO_URL.split("/")[-1].replace(".git", "")
    if not os.path.exists(repo_name):
        print(f"📦 Proje GitHub'dan çekiliyor: {repo_name}...")
        !git clone $REPO_URL
    else:
        print("📂 Proje klasörü zaten var.")
    
    %cd $repo_name
else:
    print("⚠️ Repo URL girilmedi, mevcut dizin kullanılıyor.")

# ---------------------------------------------------------
# 2. Google Drive Bağlantısı (Veri Saklama)
# ---------------------------------------------------------
if USE_GOOGLE_DRIVE:
    print("☁️ Google Drive bağlanıyor...")
    drive.mount('/content/drive')
    
    # Drive'da veri klasörü oluştur
    DRIVE_DATA_PATH = "/content/drive/MyDrive/Mistral_Mail_Data"
    if not os.path.exists(DRIVE_DATA_PATH):
        os.makedirs(DRIVE_DATA_PATH)
        print("📁 Drive'da yeni veri klasörü oluşturuldu.")
    
    # --- A. ChromaDB (Vektör Veritabanı) ---
    # Eğer yerel klasörde boş bir rag_db oluştuysa sil, Drive'dakini bağla
    if os.path.exists("rag_db") and not os.path.islink("rag_db"):
        !rm -rf rag_db
    
    # Sembolik link oluştur (App -> Drive'a yazar)
    if not os.path.exists("rag_db"):
        # Drive'da rag_db yoksa oluşturur, varsa onu kullanır
        if not os.path.exists(f"{DRIVE_DATA_PATH}/rag_db"):
             os.makedirs(f"{DRIVE_DATA_PATH}/rag_db")
        !ln -s "$DRIVE_DATA_PATH/rag_db" "rag_db"
        print("✅ Veritabanı (RAG) Drive'a bağlandı.")

    # --- B. Feedback Veriseti (JSONL) ---
    if os.path.exists("dataset_feedback.jsonl") and not os.path.islink("dataset_feedback.jsonl"):
        !rm dataset_feedback.jsonl
        
    if not os.path.exists("dataset_feedback.jsonl"):
        # Eğer Drive'da dosya yoksa boş oluştur
        if not os.path.exists(f"{DRIVE_DATA_PATH}/dataset_feedback.jsonl"):
            !touch "$DRIVE_DATA_PATH/dataset_feedback.jsonl"
        !ln -s "$DRIVE_DATA_PATH/dataset_feedback.jsonl" "dataset_feedback.jsonl"
        print("✅ Geri bildirim dosyası Drive'a bağlandı.")
        
else:
    print("⚠️ Drive kullanılmıyor. Oturum kapanınca veriler SİLİNECEK.")

# ---------------------------------------------------------
# 3. Kütüphaneleri Kur
# ---------------------------------------------------------
print("📦 Kütüphaneler kuruluyor...")
!pip install -q -r requirements.txt
!pip install -q pyngrok

print("✅ Kurulum tamamlandı.")

In [ ]:
# @title 2. Uygulamayı Başlat 🚀
from pyngrok import ngrok
import time

# Ngrok Ayarla
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# Varsa eski tünelleri kapat
ngrok.kill()

# Streamlit'i arka planda çalıştır
get_ipython().system_raw('streamlit run app.py &')

print("⏳ Streamlit başlatılıyor...")
time.sleep(5)

# Tüneli aç
try:
    public_url = ngrok.connect(8501).public_url
    print(f"\n🎉 UYGULAMA HAZIR! Tıklayın: {public_url}\n")
except Exception as e:
    print(f"Hata: {e}")

# surekli calissin diye dongu
while True:
    time.sleep(10)